Каким бы хорошим не был Python, есть у него проблема известная все разработчикам — скорость.
Оценку быстродействия удобно производить с помощью модуля time. Метод time.time() возвращает число секунд, прошедших с 00:00:00 1 января 1970 года. Вычисляя разницу между конечным и начальным временем получаем время работы нашего кода.

In [10]:
import time
start = time.time() 
sum=0
for i in range(25000):
    sum+=i
end = time.time()                                          
print((end - start)*1000, "ms") 

9.999752044677734 ms


Напишем тестовую функцию, выполняющую расчет расстояния между парами точек, хранящимися в массивах P и Q.

In [13]:
import numpy as np
import math
def get_R(p, q):
    R = np.empty((p.shape[0], q.shape[1]))

    for i in range(p.shape[0]):
        for j in range(q.shape[1]):
            rx = p[i, 0] - q[0, j]
            ry = p[i, 1] - q[1, j]
            rz = p[i, 2] - q[2, j]
            R[i, j] = math.sqrt(rx * rx + ry * ry + rz * rz)

    return R

Напишем программу для тестирования времени работы нашей функции

In [17]:
N=100
a=np.zeros((N,N))
b=np.zeros((N,N))

start = time.time()                                        
get_R(a,b)
end = time.time()                                          
print((end - start)*1000, "ms") 

30.00020980834961 ms


Увеличим размерность массива в 10 раз

In [19]:
N=1000
a=np.zeros((N,N))
b=np.zeros((N,N))

start = time.time()                                        
get_R(a,b)
end = time.time()                                          
print((end - start)*1000, "ms") 

2240.0033473968506 ms


И увидим, что время работы пропорционально квадрату N. То есть при N равном 10000, время счета составит 200 секунд.

Попробуем воспользоваться средствами библиотеки Numpy

In [21]:
def get_R_numpy(p, q):
    Rx = p[:, 0:1] - q[0:1]
    Ry = p[:, 1:2] - q[1:2]
    Rz = p[:, 2:3] - q[2:3]
    R = 1 / (1 + np.sqrt(Rx * Rx + Ry * Ry + Rz * Rz))
    return R

In [53]:
N=1000
a=np.zeros((N,N))
b=np.zeros((N,N))

start = time.time()                                        
get_R_numpy(a,b)
end = time.time()                                          
print((end - start)*1000, "ms") 

50.00019073486328 ms


Уже лучше, но еще не очень хорошо. Попробуем восспользоваться библиотекой Numba.
Для начала, попробуем решить задачу "в лоб". То есть просто укажем, что необходимо скомпилировать функцию целиком.

In [54]:
import numba     

In [55]:
@numba.jit(nopython=True)
def get_R_numba_simble(p, q):
    R = np.empty((p.shape[0], q.shape[1]))

    for i in range(p.shape[0]):
        for j in range(q.shape[1]):
            rx = p[i, 0] - q[0, j]
            ry = p[i, 1] - q[1, j]
            rz = p[i, 2] - q[2, j]

            R[i, j] = 1 / (1 + math.sqrt(rx * rx + ry * ry + rz * rz))

    return R

In [56]:
N=1000
a=np.zeros((N,N))
b=np.zeros((N,N))

start = time.time()                                        
get_R_numba_simble(a,b)
end = time.time()                                          
print((end - start)*1000, "ms") 

170.0000762939453 ms


Получилось неплохо, но можно еще лучше, если явно указать тип для возвращаемого значения и параметров функции.

In [48]:
@numba.jit(numba.float64[:, :] (numba.float64[:, :], numba.float64[:, :]), nopython=True)
def get_R_numba_mp(p, q):
    R = np.empty((p.shape[0], q.shape[1]))

    for i in range(p.shape[0]):
        for j in range(q.shape[1]):
            rx = p[i, 0] - q[0, j]
            ry = p[i, 1] - q[1, j]
            rz = p[i, 2] - q[2, j]

            R[i, j] = 1 / (1 + math.sqrt(rx * rx + ry * ry + rz * rz))

    return R

In [49]:
N=1000
a=np.zeros((N,N))
b=np.zeros((N,N))

start = time.time()                                        
get_R_numba_mp(a,b)
end = time.time()                                          
print((end - start)*1000, "ms") 

10.000228881835938 ms


И, наконец, добавим возможность считать внешний цикл параллельно. Для этого используем флаг parallel=True, и внешний цикл for i in numba.prange вместо for i in range

In [ ]:
@numba.jit(numba.float64[:, :] (numba.float64[:, :], numba.float64[:, :]), nopython=True, parallel=True)
def get_R_numba_mp(p, q):
    R = np.empty((p.shape[0], q.shape[1]))

    for i in numba.prange(p.shape[0]):
        for j in range(q.shape[1]):
            rx = p[i, 0] - q[0, j]
            ry = p[i, 1] - q[1, j]
            rz = p[i, 2] - q[2, j]

            R[i, j] = 1 / (1 + math.sqrt(rx * rx + ry * ry + rz * rz))

    return R

Последний вариант не будет запускаться на старых 32-разрядных операционных системах.

Итак, мы имеем быстродействие, сравнимое с большинством компиляторных языков, таких как C++, при сохранении синтаксиса языка Python. Конечно такая программа будет медленнее запускаться, поскольку компиляция кода, фактически, происходит при каждом запуске.